<a href="https://colab.research.google.com/github/flych3r/IA025_2022S1/blob/main/ex07/matheus_xavier/IA025_A07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Matheus Xavier Sampaio - 220092'
print(f'Meu nome é {nome}')

Meu nome é Matheus Xavier Sampaio - 220092


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
%pip install -qqq transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qqq wandb

Note: you may need to restart the kernel to use updated packages.


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [ ]:
# Check which GPU we are using
!nvidia-smi

Mon May 16 11:47:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List
from transformers import AutoTokenizer


def tokenize(text: str, tokenizer: AutoTokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer: AutoTokenizer, context_size: int):
        # Escreva seu código aqui
        self.tokenizer = tokenizer
        self.context_size = context_size
        self.n_grams, self.next_words = self._transform_texts(texts)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.next_words)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return self.n_grams[idx], self.next_words[idx]
    
    def _transform_texts(self, texts):
        tokens = (
            tokenize(text, self.tokenizer) for text in tqdm(texts)
        )
        tokens = [
            (tkn[i:i + self.context_size], tkn[i + self.context_size]) 
                for tkn in tokens
            for i in range(len(tkn) - (self.context_size))
        ]
        n_grams = [tkn[0] for tkn in tokens]
        next_words = [tkn[1] for tkn in tokens]
        return torch.LongTensor(n_grams), torch.LongTensor(next_words)

### Teste se sua implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor([
    [ 3396, 10303,   125],
    [ 1660,  5971,   785],
    [ 5971,   785,   125],
    [  785,   125,  1847],
    [  125,  1847, 13779]
])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')

assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


### Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-16 11:47:11--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 142.251.107.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   108MB/s    in 1.1s    

2022-05-16 11:47:13 (108 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [ ]:
# Load datasets
context_size = 9

texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]
valid_examples = int(len(texts) * 0.2)
test_examples = int(len(texts) * 0.2)

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

  0%|          | 0/15000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 17196588
valid examples: 5119032
test examples: 5609121


## Modelo de linguagem

In [ ]:
class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size: int, context_size: int, embedding_dim: int, hidden_size: int):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.emb = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.hidden = torch.nn.Linear(in_features=context_size * embedding_dim, out_features=hidden_size)
        self.output = torch.nn.Linear(in_features=hidden_size, out_features=vocab_size)

    def forward(self, inputs: torch.Tensor):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """
        # Escreva seu código aqui.
        out = self.emb(inputs)
        out = out.flatten(start_dim=1)
        out = self.hidden(out)
        out = torch.nn.functional.relu(out)
        out = self.output(out)
        return out

### Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5824098


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    # Escreva seu código aqui.
    loss = torch.nn.functional.cross_entropy(logits, target)
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              30890
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
import wandb
from copy import deepcopy

wandb.init(project="bengio-lm", anonymous="allow")

wandb: Currently logged in as: flych3r. Use `wandb login --relogin` to force relogin


In [ ]:
max_examples = 500_000_000
eval_every_steps = 10_000
lr = 3e-5
batch_size = 4096 * 3

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=batch_size)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0

best_ppl = torch.inf
best_model = deepcopy(model.state_dict())

wandb.watch(model, log_freq=100)
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for input, target in train_loader:
        model.train()
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)

        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)
            train_ppl = np.exp(train_loss)
    
            model.eval()
            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader
                ]))
            
            wandb.log({
                "train/loss": loss,
                "train/perplexity": train_ppl,
                "eval/perplexity": valid_ppl
            }, step=step)
            if valid_ppl < best_ppl:
                best_ppl = valid_ppl
                best_model = deepcopy(model.state_dict())
                torch.save(best_model, 'best_model.pth')
                
                artifact = wandb.Artifact(
                    'model',
                    type='model',
                    metadata={
                        "step": step,
                        "train_loss": train_loss,
                        "train_perplexity": train_ppl,
                        "valid_perplexity": valid_ppl
                    }
                )
                artifact.add_file('best_model.pth')
                wandb.run.log_artifact(artifact)

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        pbar.update(len(input))
        if n_examples >= max_examples:
            break

model.load_state_dict(best_model)

  0%|          | 0/500000000 [00:00<?, ?it/s]

0 steps; 0 examples so far; train ppl: 30588.04, valid ppl: 30488.00
1000 steps; 12288000 examples so far; train ppl: 4635.47, valid ppl: 2047.36
2000 steps; 24576000 examples so far; train ppl: 1736.74, valid ppl: 1516.08
3000 steps; 36864000 examples so far; train ppl: 1342.08, valid ppl: 1222.12
4000 steps; 49152000 examples so far; train ppl: 1103.76, valid ppl: 1036.56
5000 steps; 61440000 examples so far; train ppl: 947.22, valid ppl: 911.11
6000 steps; 73728000 examples so far; train ppl: 838.23, valid ppl: 819.33
7000 steps; 86016000 examples so far; train ppl: 755.27, valid ppl: 747.76
8000 steps; 98304000 examples so far; train ppl: 689.13, valid ppl: 690.20
9000 steps; 110592000 examples so far; train ppl: 635.29, valid ppl: 642.09
10000 steps; 122880000 examples so far; train ppl: 590.50, valid ppl: 600.98
11000 steps; 135168000 examples so far; train ppl: 551.14, valid ppl: 565.24
12000 steps; 147456000 examples so far; train ppl: 517.04, valid ppl: 534.18
13000 steps; 159

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 263.5974020288667


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

model.eval()
for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com a
Eu gosto de comer pizza pois me faz com a minha
Eu gosto de comer pizza pois me faz com a minha vida
Eu gosto de comer pizza pois me faz com a minha vida,
Eu gosto de comer pizza pois me faz com a minha vida, mas
Eu gosto de comer pizza pois me faz com a minha vida, mas não
Eu gosto de comer pizza pois me faz com a minha vida, mas não me
Eu gosto de comer pizza pois me faz com a minha vida, mas não me engan
Eu gosto de comer pizza pois me faz com a minha vida, mas não me engano


In [ ]:
wandb.finish()

eval/perplexity,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/perplexity,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/perplexity,266.91794
train/loss,5.47605
train/perplexity,229.90546
